# Circuit Deep Dive Analysis

Detailed exploration of key Formula 1 circuits and their characteristics.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from src.circuit_analysis import analyze_circuit
from src.circuit_reporting import compare_circuits

sns.set_style('whitegrid')
%matplotlib inline

## Load Data

In [ ]:
# Load race data
train = pd.read_csv('../data/processed/train.csv')

# Load cluster info
try:
    clusters = pd.read_csv('../results/clustering/circuit_clusters.csv')
except:
    clusters = None

print(f"Loaded {len(train)} race results")
print(f"Circuits: {train['circuit'].nunique()}")

## Selected Circuits for Deep Dive

We've selected diverse circuits representing different racing characteristics:

In [ ]:
selected_circuits = {
    'Monte Carlo': 'Ultimate street circuit - minimal overtaking',
    'Monza': 'Temple of speed - low downforce, high speeds',
    'Spa-Francorchamps': 'Classic track - weather variable, long lap',
    'Sakhir': 'Modern facility - good racing',
    'Marina Bay': 'Street circuit with overtaking opportunities',
    'Silverstone': 'High-speed, classic layout'
}

# Analyze all circuits
all_analyses = {}
for circuit in train['circuit'].unique():
    analysis = analyze_circuit(train, circuit, clusters)
    if analysis is not None:
        all_analyses[circuit] = analysis

## Circuit Comparison Overview

In [ ]:
comparison_metrics = []

for circuit, description in selected_circuits.items():
    if circuit in all_analyses:
        analysis = all_analyses[circuit]
        
        metrics = {
            'Circuit': circuit,
            'Type': description.split('-')[0].strip(),
            'Pole Win Rate': analysis['notable_stats']['pole_win_rate'],
            'Avg Pos Change': abs(analysis['position_change_stats']['mean_change']),
            'Overtaking Score': analysis['notable_stats']['overtaking_score'],
            'DNF Rate': analysis['notable_stats']['dnf_rate'],
            'Cluster': analysis['notable_stats'].get('cluster_name', 'Unknown')
        }
        comparison_metrics.append(metrics)

comparison_df = pd.DataFrame(comparison_metrics)
comparison_df

## Monaco vs Monza: The Extremes

In [ ]:
if 'Monte Carlo' in all_analyses and 'Monza' in all_analyses:
    monaco_monza = compare_circuits('Monte Carlo', 'Monza', all_analyses)
    print("\nMonaco vs Monza Comparison:")
    print(monaco_monza)

In [ ]:
# Visual comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

if 'Monte Carlo' in all_analyses:
    monaco_stats = all_analyses['Monte Carlo']['grid_statistics']
    axes[0].plot(monaco_stats['grid_position'], monaco_stats['avg_finish'], 
                'ro-', linewidth=2, markersize=8, label='Avg Finish')
    axes[0].plot([1, 20], [1, 20], 'k--', alpha=0.3, label='No Change')
    axes[0].set_xlabel('Grid Position')
    axes[0].set_ylabel('Average Finish Position')
    axes[0].set_title('Monte Carlo (Monaco) - Grid Matters', fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)

if 'Monza' in all_analyses:
    monza_stats = all_analyses['Monza']['grid_statistics']
    axes[1].plot(monza_stats['grid_position'], monza_stats['avg_finish'], 
                'bo-', linewidth=2, markersize=8, label='Avg Finish')
    axes[1].plot([1, 20], [1, 20], 'k--', alpha=0.3, label='No Change')
    axes[1].set_xlabel('Grid Position')
    axes[1].set_ylabel('Average Finish Position')
    axes[1].set_title('Monza - More Opportunity', fontweight='bold')
    axes[1].legend()
    axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Success Probabilities Analysis

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

for circuit in ['Monte Carlo', 'Monza', 'Spa-Francorchamps', 'Silverstone']:
    if circuit in all_analyses:
        probs = all_analyses[circuit]['success_probabilities']
        x_pos = range(1, min(11, len(probs) + 1))
        podium_probs = probs['podium_prob'].values[:10]
        ax.plot(x_pos, podium_probs, 'o-', label=circuit, linewidth=2, markersize=6)

ax.set_xlabel('Grid Position', fontsize=11)
ax.set_ylabel('Podium Probability', fontsize=11)
ax.set_title('Podium Chances by Grid Position Across Circuits', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)
plt.show()

## Historical Evolution

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Pole win rate evolution
for circuit in selected_circuits.keys():
    if circuit in all_analyses:
        trends = all_analyses[circuit]['historical_trends']
        if len(trends) > 1 and 'pole_win_rate' in trends.columns:
            axes[0].plot(trends.index, trends['pole_win_rate'], 
                        'o-', label=circuit, linewidth=2, markersize=6)

axes[0].set_ylabel('Pole Win Rate')
axes[0].set_title('Pole Position Advantage Evolution', fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# DNF rate evolution
for circuit in selected_circuits.keys():
    if circuit in all_analyses:
        trends = all_analyses[circuit]['historical_trends']
        if len(trends) > 1 and 'is_dnf_mean' in trends.columns:
            axes[1].plot(trends.index, trends['is_dnf_mean'], 
                        'o-', label=circuit, linewidth=2, markersize=6)

axes[1].set_xlabel('Year')
axes[1].set_ylabel('DNF Rate')
axes[1].set_title('Reliability Evolution', fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Circuit Insights

In [ ]:
for circuit, description in selected_circuits.items():
    if circuit not in all_analyses:
        continue
        
    analysis = all_analyses[circuit]
    
    print(f"\n{'='*70}")
    print(f"CIRCUIT: {circuit.upper()}")
    print(f"{'='*70}")
    print(f"\nDescription: {description}")
    
    print(f"\nKey Characteristics:")
    print(f"  • Cluster: {analysis['notable_stats'].get('cluster_name', 'Unknown')}")
    print(f"  • Pole Win Rate: {analysis['notable_stats']['pole_win_rate']:.1%}")
    print(f"  • Overtaking Score: {analysis['notable_stats']['overtaking_score']:.2f}")
    print(f"  • DNF Rate: {analysis['notable_stats']['dnf_rate']:.1%}")
    print(f"  • Position Change StdDev: {analysis['position_change_stats']['std_change']:.2f}")
    
    print(f"\nNotable Records:")
    print(f"  • Lowest Winner Grid: P{analysis['notable_stats']['lowest_winner_grid']}")
    print(f"  • Highest Podium Grid: P{analysis['notable_stats']['highest_podium_grid']}")
    print(f"  • Max Positions Gained: +{analysis['notable_stats']['max_positions_gained']}")
    
    if analysis['notable_stats']['pole_win_rate'] > 0.6:
        print(f"\n  ⚠️  High grid position advantage - qualifying is crucial")
    elif analysis['notable_stats']['pole_win_rate'] < 0.3:
        print(f"\n  ✅ Low grid position advantage - race day opportunities")
    
    if analysis['position_change_stats']['std_change'] > 4:
        print(f"  🎲 High variance - unpredictable racing")
    elif analysis['position_change_stats']['std_change'] < 2:
        print(f"  📊 Low variance - processional racing")

## Overall Statistics Summary

In [ ]:
avg_pole_win = np.mean([a['notable_stats']['pole_win_rate'] for a in all_analyses.values()])
avg_pos_change = np.mean([abs(a['position_change_stats']['mean_change']) for a in all_analyses.values()])
avg_dnf = np.mean([a['notable_stats']['dnf_rate'] for a in all_analyses.values()])

print("\nOverall F1 Circuit Statistics:")
print(f"  • Average Pole-to-Win Rate: {avg_pole_win:.1%}")
print(f"  • Average Position Changes: ±{avg_pos_change:.2f}")
print(f"  • Average DNF Rate: {avg_dnf:.1%}")

most_processional = max(all_analyses.items(), key=lambda x: x[1]['notable_stats']['pole_win_rate'])
most_chaotic = max(all_analyses.items(), key=lambda x: x[1]['position_change_stats']['std_change'])
safest = min(all_analyses.items(), key=lambda x: x[1]['notable_stats']['dnf_rate'])

print(f"\nExtreme Circuits:")
print(f"  • Most Processional: {most_processional[0]}")
print(f"  • Most Chaotic: {most_chaotic[0]}")
print(f"  • Safest: {safest[0]}")